In [338]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [339]:
data = pd.read_csv("train.csv")
y = data['SalePrice']
X = data.drop('SalePrice', axis=1)
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [340]:
X.drop('Id', axis=1)
numeric_cols = list(X.select_dtypes(include=np.number).columns)
X_num = X[numeric_cols]
X_num.columns = X[numeric_cols].columns
X_num.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


## First model will use only numeric values and missing values will be imputed with SimpleImputer

In [341]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

def get_train_test_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_test.columns = X_test.columns
    
    X_train, X_test = imputed_X_train, imputed_X_test
    
    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
    y_test = torch.tensor(y_test.values, dtype=torch.float32).squeeze()
    return X_train, X_test, y_train, y_test

In [342]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)
X_num = pd.DataFrame(my_imputer.transform(X_num))

In [343]:
class ModelV1(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=1),
            # nn.Linear(in_features=10, out_features=10),
            # nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [344]:
model1 = ModelV1(in_features=37)

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model1.parameters(), lr=0.0001)

In [345]:
model1.eval()
y_pred = model1(X_train).squeeze()
y_pred

tensor([668.1719, 341.4703, 610.5351,  ..., 294.5693, 132.4700, 317.1259],
       grad_fn=<SqueezeBackward0>)

In [346]:
def train_test(model:torch.nn.Module, loss_fn:torch.nn.Module, optimizer:torch.optim.Optimizer, epochs:int, X_train, X_test, y_train, y_test):
    
    for epoch in range(epochs):
        model.train()
        y_preds = model(X_train).squeeze()
        loss = loss_fn(y_preds, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with (torch.inference_mode()):
            test_pred = model(X_test).squeeze()
            test_loss = loss_fn(test_pred, y_test)
        if epoch % 50 == 0:
            print("----------------------------")
            print(f"Epoch {epoch}:")
            print(f"Train loss: {loss} | Test loss: {test_loss}")

In [347]:
train_test(model=model1, loss_fn=loss_fn, optimizer=optimizer, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 181328.359375 | Test loss: 163381.640625
----------------------------
Epoch 50:
Train loss: 60191.76171875 | Test loss: 57289.18359375
----------------------------
Epoch 100:
Train loss: 55971.21875 | Test loss: 53504.421875
----------------------------
Epoch 150:
Train loss: 52445.51171875 | Test loss: 50371.92578125
----------------------------
Epoch 200:
Train loss: 49821.9921875 | Test loss: 48042.23828125
----------------------------
Epoch 250:
Train loss: 47865.40625 | Test loss: 46192.59765625
----------------------------
Epoch 300:
Train loss: 46356.34765625 | Test loss: 44731.65234375
----------------------------
Epoch 350:
Train loss: 45232.9375 | Test loss: 43667.5703125
----------------------------
Epoch 400:
Train loss: 44301.7421875 | Test loss: 42774.76171875
----------------------------
Epoch 450:
Train loss: 43458.4453125 | Test loss: 41960.04296875
----------------------------
Epoch 500:
Train loss: 42724.9375 | Test l

## This results are not great, let's figure out which 5 data columns have the most corelation with SalePrice and use only them

In [348]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [349]:
discrete_features = X_num.dtypes == int

mi_scores = make_mi_scores(X_num, y, discrete_features=discrete_features)
mi_scores[:7]

4     0.559256
16    0.480458
6     0.370888
12    0.365883
27    0.363714
26    0.359286
13    0.309015
Name: MI Scores, dtype: float64

## Let's now use only this 7 features in a Model and see if there is any difference in performance

In [350]:
model2 = ModelV1(in_features=7)
lista = list(mi_scores[:7].index)
for i in lista:
    i -= 1
X_num_2 = X_num[lista]
X_train, X_test, y_train, y_test = get_train_test_data(X_num_2, y, test_size=0.2)
optimizer_2 = torch.optim.SGD(params=model2.parameters(), lr=0.001)

In [351]:
train_test(model=model2, loss_fn=loss_fn, optimizer=optimizer_2, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 182127.25 | Test loss: 173775.984375
----------------------------
Epoch 50:
Train loss: 37753.9921875 | Test loss: 41764.93359375
----------------------------
Epoch 100:
Train loss: 36077.8359375 | Test loss: 39832.43359375
----------------------------
Epoch 150:
Train loss: 34681.5859375 | Test loss: 38329.70703125
----------------------------
Epoch 200:
Train loss: 33595.33984375 | Test loss: 37123.32421875
----------------------------
Epoch 250:
Train loss: 32751.048828125 | Test loss: 36143.328125
----------------------------
Epoch 300:
Train loss: 32031.763671875 | Test loss: 35308.5546875
----------------------------
Epoch 350:
Train loss: 31488.30078125 | Test loss: 34656.73828125
----------------------------
Epoch 400:
Train loss: 31080.380859375 | Test loss: 34142.70703125
----------------------------
Epoch 450:
Train loss: 30755.951171875 | Test loss: 33785.98046875
----------------------------
Epoch 500:
Train loss: 30483.623

## Now let's try to normalize data and see whether it improves our result or not

In [352]:
X_num_3 = X_num_2
y_normalized = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(y, dtype=torch.float32).unsqueeze(dim=1)))
X_train, X_test, y_train, y_test = get_train_test_data(X_num_3, y_normalized, test_size=0.2)
X_train, X_test = torch.nn.functional.normalize(X_train), torch.nn.functional.normalize(X_test)

In [355]:
model3 = ModelV1(in_features=7)
optimizer_3 = torch.optim.SGD(params=model3.parameters(), lr=0.0001)

In [356]:
train_test(model=model3, loss_fn=loss_fn, optimizer=optimizer_3, epochs=2001, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 0.4545436501502991 | Test loss: 0.45776137709617615
----------------------------
Epoch 50:
Train loss: 0.444738507270813 | Test loss: 0.44794961810112
----------------------------
Epoch 100:
Train loss: 0.43493330478668213 | Test loss: 0.43813785910606384
----------------------------
Epoch 150:
Train loss: 0.42512819170951843 | Test loss: 0.4283260703086853
----------------------------
Epoch 200:
Train loss: 0.41532304883003235 | Test loss: 0.41851434111595154
----------------------------
Epoch 250:
Train loss: 0.40551790595054626 | Test loss: 0.4087026119232178
----------------------------
Epoch 300:
Train loss: 0.39571279287338257 | Test loss: 0.398890882730484
----------------------------
Epoch 350:
Train loss: 0.38590767979621887 | Test loss: 0.38907915353775024
----------------------------
Epoch 400:
Train loss: 0.3761025667190552 | Test loss: 0.37926745414733887
----------------------------
Epoch 450:
Train loss: 0.366297483444213